<a href="https://colab.research.google.com/github/haydenclose/Cloud_based_Oil_Detection/blob/main/Cloud_Based_Analysis_of_Oil_Spills_from_Wrecks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cloud Based Analysis of Oil Spills from Wrecks

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1 Set-up of workspace
Import Earth Engine API (`import ee`).
Additionally, import useful python packages from examplar scripts (need to remove redundant ones after)

In [ ]:
import ee
!pip install geemap
import geemap
!pip install geopandas
import geopandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd                                                             # Useful package to read in csv's etc...
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
%matplotlib inline
import ipyleaflet
import ipywidgets as widgets

Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell. Authorisation last a week.

In [ ]:
# Trigger the authentication flow.. Only need to do once a week
ee.Authenticate()

# Initialize the library
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=4_7TYjPZf6-URqk4QOd1jLNlO-Pe4Zk9N2TCjB0W7xU&tc=oCaXHss_YQcm7i7IOpS6nCenR5xvbcRggIUC9bG7KVQ&cc=meLPJPXTmRq0J5VEWliz2R-2oimwQF28LFjoepZCn2E

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AbUR2VOapAlYJ9MpJotr0GQ3zMM4yfrmhH1Wqnu6mDRBisZ7Vh7pgJT0rzw

Successfully saved authorization token.


## 2 Interactive Maps

## 2.1 Manual creation of maps using Folium package

The [`folium`](https://python-visualization.github.io/folium/)
library can be used to display `ee.Image` objects on an interactive
[Leaflet](https://leafletjs.com/) map. Folium has no default
method for handling tiles from Earth Engine, so one must be defined
and added to the `folium.Map` module before use.

The following cell provides an example of adding a method for handing Earth Engine
tiles and using it to display an elevation model to a Leaflet map

In [ ]:
# Import the Folium library.
import folium                                                                   # Used for Interactive mapping

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):                      # Define function and variables
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)                  # Method to display the  map
  folium.raster_layers.TileLayer(                                               # Add one of the mase maps and add options below
    tiles = map_id_dict['tile_fetcher'].url_format,                             # Not sure what this particularly does
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>', # or what this does
    name = name,                                                                # Adds the layer name to the radio buttons rather than web address
    overlay = True,                                                             # Allow layers to overlay upon each
    control = True                                                              # Allows the user to turn on and off layers
  ).add_to(self)                                                                # Allows layers to be added after this base layer

# Add EE drawing method to folium, to be able to add other layers to this base map (see examplar below)
folium.Map.add_ee_layer = add_ee_layer                                         

from folium.plugins import Draw                                                 # Toolbox for adding the toolbar
draw = Draw(export=True)                                                        # Adds toolbars to folium plots

print('This sets up the base parameters of the map with no output')             # Prints a statement so you know its ran


Now lets make an examplar interactive map just with the base layer

In [ ]:
lat, lon = 52, 0                                                                # Define the Lat and Lon to centre the map on (UK)
my_map = folium.Map(location=[lat, lon], zoom_start=7)                          # Create the map zoomed to level 7 on our defined Lat and Lon (not yet ouputted)
draw.add_to(my_map)                                                             # Add the draw toolbar to the map
display(my_map)                                                                 # Display the map

## 2.3 Annotating map with points
We have a list of wreck locations (stored on sharePoint under 'Data for Google Colab'). I have uploaded the file called 'Wreck Database_V2.2.xls' to my google drive, you will need to do the same or you can use the navigation panel to the left `<-` click the folder icon and then upload a file (first icon along the top) but this will only be kept for an active session so will need to be uploaded each time. 

Now we use pandas to read the excel file and the `Wreck.head()` to view the first few rows of the dataframe. Note the ***magic wand*** button that allows for the interaction with the data



In [165]:
Wrecks = pd.read_excel('/content/drive/MyDrive/Wreck Database_V2.3.xls')      # Loads the xls file from your personal google drive
Wrecks.head()                                                                   # Displays the data to interact with

,Owner,Wreck_ID,Catergory,Latitude,Longitude,Depth,Cargo
0,MOD,SS DERBENT,Priority,53.473390,-4.235660,45m,3700 tons of fuel oil
1,MOD,HMS PRINCE OF WALES,Priority,3.517583,104.464383,~68m,NaN
2,MOD,HMS REPULSE,Priority,3.620619,104.345181,~68m,NaN
3,MOD,RFA WAR MEHTAR,Priority,52.605000,2.148330,Between 26.5 m and 40 m with 2-3 m scour.,7000 tonnes Admiralty fuel oil
4,MOD,RFA ATHELSTANE,Priority,7.332520,81.939600,42m,6096 tonnes Admiralty fuel oil


### 2.31 Adding the wrecks to the map
Here we use a little `for` loop to go through each row of the wreck dataframe and catergorise by priority.

This method also adds the wrecks to the `Folium.FeatureGroup`  so we can turn on and off the wrecks we do and dont want in the feature control panel.

Finally we can add a `popup`, this can be an extensive table but for now to keep simple it is just the wrecks name.

In [ ]:
Point_map = folium.Map(location=[lat, lon], zoom_start=7)                       # Base map with zoomed location and level.
for Wreck_group, Wreck in Wrecks.groupby('Catergory'):                          # For loop to group the wrecks together by priority
    feature_group = folium.FeatureGroup(Wreck_group)                            # Add the wreck groups to the feature control panel
    for row in Wreck.itertuples():                                              # Loop through the individual wrecks to plot
        folium.CircleMarker(location=[row.Latitude, row.Longitude],             # Adds circle marker by lat and long
                            popup=row.Wreck_ID,                                 # If click a point, tells us the name of the wreck
                            radius = 3,                                         # Size of point
                            fill_opacity=1).add_to(feature_group)               # Colour of outline by catergory
    feature_group.add_to(Point_map)                                             # Adds the points to the map 

folium.LayerControl().add_to(Point_map)                                         # Add the layer control to the map
Point_map                                                                       # Plots the map

## 2.4 Using interactive geemap
The above is a long winded process and extensive code. Fortuneatly there an amazing package called `geemap` that wraps up alot of different functions  based on the previous `folium` package and leaflet into nice interactive mapping utility. Notice the below the one simple line to create everthing and the toolbox allows for interactive changes to base map etc... without hard coding it.

See below for the basic output and explanation

*   Has tool bar on the left that can search by name/address, lat-lon or data
*   Draw toolbar to add polygons, points etc...
*   On the right have a little spanner that has a lot of useful tools see https://geemap.org/
*   At the top if click the symbol next to the spanner can toggle layers on and off and adjust transparency

**We will be using  `geemap` from here onwards but can default back `folium` to customise ourselves if we think it is required**






In [ ]:
BasicMap = geemap.Map()                                                         # Basic interactive map with no added data
BasicMap                                                                        # Display the basic map

### 2.41 Add Points to geemap

Add ponts from a dataframe with a simple piece of code `Map.add_points_from_xy(df, x="", y="")`

This also add the data as a popup when clicked
 
**NOTE. Found a little bug, need to load the wreck data in again else get duplicate information in the wreck popup** 

In [ ]:
PointsGeeMap = geemap.Map()                                                     # Basic map to draw onto
Wrecks = pd.read_excel('/content/drive/MyDrive/Wreck Database_V2.3.xls')      # Loads the xls file from your personal google drive
PointsGeeMap.add_points_from_xy(Wrecks, x="Longitude", y="Latitude")            # Simple line to add data (note can click the points for popup of data)
PointsGeeMap                                                                    # Displays the map

### 2.42 Sentinel-1 imagery example
Example from https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S1_GRD

note this uses the `VV` band and `IW` instrument mode. Additionally, it creates a mean from the different months to identify change. 

Ideally we just need to look at one timestamp as the oil slicks will move with the tide (if any). Note the data is missing for the area of interest of Maylasia for the HMS Repulse and Prince of Wales.

In [ ]:
Wrecks = pd.read_excel('/content/drive/MyDrive/Wreck Database_V2.3.xls')       
SentExampleMap = geemap.Map()
imgVV = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .select('VV')

def func_skh(image):
          edge = image.lt(-30.0)
          maskedImage = image.mask().And(edge.Not())
          return image.updateMask(maskedImage) \
        .map(func_skh)

desc = imgVV.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
asc = imgVV.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

spring = ee.Filter.date('2015-03-01', '2016-05-20')
lateSpring = ee.Filter.date('2015-04-21', '2016-05-10')
summer = ee.Filter.date('2015-06-11', '2016-08-31')

descChange = ee.Image.cat(
        desc.filter(spring).mean(),
        desc.filter(lateSpring).mean(),
        desc.filter(summer).mean())

ascChange = ee.Image.cat(
        asc.filter(spring).mean(),
        asc.filter(lateSpring).mean(),
        asc.filter(summer).mean())

SentExampleMap.setCenter(104.3, 3.6, 10)
SentExampleMap.addLayer(ascChange, {'min': -25, 'max': 5}, 'Multi-T Mean ASC', True)
SentExampleMap.addLayer(descChange, {'min': -25, 'max': 5}, 'Multi-T Mean DESC', True)
SentExampleMap.add_points_from_xy(Wrecks, x="Longitude", y="Latitude")  
SentExampleMap

#3 Find out which orbits and flights paths cover our Area of Interest (AoI)



### 3.1 **HMS Repulse** and **HMS Prince of Wales** just of Malaysia.

In [ ]:
OrbitsMap = geemap.Map()                                                                            # Base map

# Area of Interest of Malaysia
geometry = ee.Geometry.Polygon(
  [[[104.5, 3.5],
   [104.5, 3.5],
   [104.5, 3.7],
   [104.3, 3.7],
   [104.3, 3.5]]], None, False)


# Get Sentinel-1 data for an arbitrary 12 day period
s1 = (ee.ImageCollection('COPERNICUS/S1_GRD').                                                      # Selects the Sentinel 1 image collection   
  filterDate('2014-04-03', '2015-05-29').                                                           # Selects only the dates we are interested in
  filterMetadata('instrumentMode', 'equals', 'IW').                                                 # Selects the instrument mode that we wont
  filterBounds(geometry))                                                                           # Selects only images that our AoI is contained within

# This function gets the relative orbits to be able to plot them
def func_xmf(f):                                                                                    # Function definition
  return f.set('platform_relorbit',                                                                 # Full description i.e. 'A_18_Descending'
    ee.String(f.get('platform_number')).cat('_').                                                   # Determine whether A or B satelite
    cat(ee.Number(f.get('relativeOrbitNumber_start')).format('%.0f')).                              # Start number of orbit i.e. 18
    cat('_').cat(f.get('orbitProperties_pass')))                                                    # determines whether satelite ascending or descending

s1 = s1.map(func_xmf)                                                                               # Extracts the relative orbits from the image collection
orbits = ee.Dictionary(s1.aggregate_histogram('platform_relorbit'))                                 # Check which sensor/relative orbit combinations we have
keys = orbits.getInfo()                                                                             # Needed to loop

for k in keys:                                                                                      # Colours the orbits,
  color = 'blue'                                                                                    # Blue = A
  if (k[0]=='B'):                                                                                   # If orbit info [0] = B then colours it
    color = 'green'                                                                                 # Green = B
  
  OrbitsMap.addLayer(ee.Image().paint(ee.FeatureCollection(s1).                                     # Add outline of image 
    filterMetadata('platform_relorbit', 'equals', k), 0, 1),                                        # extract the name for the orbit, 0, 1 = position of  A or B in the string 
    {'palette': [color]}, 'Img' + k, True)                                                          # Changes the colour and adds the name to the layer feature group

for k in keys:                                                                                      # For all images
  OrbitsMap.addLayer(s1.filterMetadata('platform_relorbit', 'equals', k).first(),                   # Add the relative orbit again
    {'bands': ['angle'], 'min': 30, 'max': 45}, 'Incidence angle: ' + k, False)                     # Plot the angle of the receiving signal. True/False whether to display or not

OrbitsMap.addLayer(ee.Image().paint(geometry, 0, 1), {'palette': ['red']}, 'AOI', True)             # Adds the area of interest to the map
OrbitsMap.centerObject(geometry, 6)                                                                 # Centerss the map 
Wrecks = pd.read_excel('/content/drive/MyDrive/Wreck Database_V2.3.xls')                            # Loads the xls file from your personal google drive
OrbitsMap.add_points_from_xy(Wrecks, x="Longitude", y="Latitude")                                   # Adds the points to the map
OrbitsMap                                                                                           # Displays the map


Map(center=[3.599998163557322, 104.39999999999922], controls=(WidgetControl(options=['position', 'transparent_…

# 4 Adding a single Sentinel-1 Image and detecting oil

So we have found out there is images available. Im we simple type s1 it gives us a drop down menu and we can then look at the dates to narrow down the image we want (need to run the code chunk above first).

There **should be easy**, but havent found the solution to extract data from the `ImageCollection` easily yet so **for now il do manually**.

In part three we found out the orbits and images available for our AOI, these can be seen by the below code `s1` the name of our image collection

In [ ]:
s1

using the above, navigate to the features to be able to paste the image name **S1A_IW_GRDH_1SDV_20150227T224655_20150227T224723_004815_005FD1_808B** into the first line e.g.
`img=ee.Image('COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20150227T224655_20150227T224723_004815_005FD1_808B').select('VV')`


In [ ]:
# Selected image from list from image collection above
img = ee.Image('COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20150227T224655_20150227T224723_004815_005FD1_808B').select('VV')
# Loads the xls file from your personal google drive
Wrecks = pd.read_excel('/content/drive/MyDrive/Wreck Database_V2.3.xls')                            

# Area of Interest of Malaysia
geometry = ee.Geometry.Polygon(
  [[[104.5, 3.5],
   [104.5, 3.5],
   [104.5, 3.7],
   [104.3, 3.7],
   [104.3, 3.5]]], None, False)

SingleImgMap = geemap.Map()                                                                         # Base map
SingleImgMap.add_points_from_xy(Wrecks, x="Longitude", y="Latitude")                                # Add wreck locations

img_params = {'bands':'VV', 'min':-25, 'max':5}                                                     # Selects the VV band and the min and max values to display

# The metadata doesnt have the date, have to extract from image name (called index)
year = ee.Image(img).getString('system:index').slice(-50, 21)                                       # Select out the year
month = ee.Image(img).getString('system:index').slice(-46, 23)                                      # Select out the month
day = ee.Image(img).getString('system:index').slice(-44, 25)                                        # Select out the day
date = ee.String(year.cat('-').cat(month).cat('-').cat(day))                                        # Format to date format e.g. 2023_01_01
date = date.getInfo()                                                                               # ee.String an object on server side so need to extract it

#Visualizing the map
SingleImgMap.addLayer(img, img_params, date,True)                                                   # Adds the image based on parameters defined
SingleImgMap.centerObject(geometry, 11)                                                             # Centers map on location 9 = full image
SingleImgMap                                                                                        # Display the map

Map(center=[3.599998163557322, 104.39999999999922], controls=(WidgetControl(options=['position', 'transparent_…

## 4.1 Adding a interactive textbox widget to the map
Adds a interactive textbox to the bottom right of the map, allows the user to type the wreck name in. We could extract it from the point layer but thought we might want to explore other wrecks and not be limited to our list and if locations not accurate. This textbox is used to populate the table with the date and size of the spill.

In [ ]:
WreckName = widgets.Text(value ='',                                                                 # Blank so we can type over
                         placeholder='Enter a wreck name',                                          # Text to prompt the user
                         description='Wreck:')                                                      # Text in front of the interactive box

WreckNameWidget = ipyleaflet.WidgetControl(widget=WreckName, position='bottomright')                # Method to add to the map
SingleImgMap.add_control(WreckNameWidget)                                                           # Adds the widget to the map

## 4.2 Delineating potential oil spills

From the image we can clearly see some potential leak from the two wrecks. This an **important step in determining what is and not oil**. We can select out pixels based on a certain thrershold and highlight them. To do this we use the inspector tool to figure out the value of the area of the spills. I have done this and it is roughly -23 or lower. You can use `lt` for lower than and `gt` for greater than and then mask (hide) the other pixels. I then highlighted these pixels in red and renamed the layer *Oil slicks*. 

In [ ]:
Oil = (img.lt(-23.5).selfMask().rename('Oil slicks'))                                               # Selects pixel values less than -23.5
SingleImgMap.addLayer(Oil, {'palette': 'FF0000'}, 'Pixels < 23.5')                                  # Add mask layer, colour red and rename

## 4.3 Converting raster to vector

If we use the smallest pixel value (10 m) then GEE doesnt like it and it takes alot of time so to help speed up and allow the highest resolution we need to draw a polygon around the oil spills on the above map. 

The below code then extracts it from the map to be used in converting the pixels into rasters `ee.FeatureCollection(SingleImgMap.draw_features)`

In [ ]:
AoI = ee.FeatureCollection(SingleImgMap.draw_features)                                              # Extracts the drawn polygon from the map

## 4.4 extracting the oil spill areas
This is **another important step in determing what is and not oil**. Above we determined the pixel value but here in the code below **`scale`** is important as refers to the image resolution, in this case the max is 10m. You can play with this and change to 50 or 100 and see each square of polygon become larger. Additionally, the **`maxPixels`** determines the maximum number of pixels GEE can deal with. If you get an error or the polygons dont load you can try increasing this or make the drawn polygon smaller. `Eightconnected` refers will include pixels in a polgon if connect on a diagonal, include for now.

The following code `Oil_Polygons.filter(ee.Filter.gt('count',25))` is particularly **crucial to determining the extent of any potential oil spills**. This filters the polygons to remove any polygons comprised of 25 pixels or less. this removes alot of the noise although some small polygons may still exist but we can get rid of them by using distance from the main oil spills.

In [ ]:
Oil_Polygons = Oil.reduceToVectors(geometry = AoI,                                                  # Polygon extracted from the map
                             scale = 10,                                                            # Resolution of data in meters
                             geometryInNativeProjection =True,                                      # Use the image projection                                 
                             maxPixels = 1e10,                                                      # Max pixels GEE will deal with                                                  
                             eightConnected =True)                                                  # Polgons will count as connect if at a diagonal
Oil_Polygons = Oil_Polygons.filter(ee.Filter.gt('count',25))                                        # Filters the polgons whic have more than 25 pixels

SingleImgMap.addLayer(ee.Image().paint(Oil_Polygons), {'palette': ['F2C80F']}, 'Oil_Polygons', True)# FE6DB6= pink


## 4.5 Extracting the data from the map
Here i extract the area of all the polygons then combine it with the date and name of the wreck to export.

In [ ]:
larea = Oil_Polygons.geometry().area(maxError = 1).getInfo()                                        # Gets the area of the total oil spill
data =[[WreckName.value,date, larea]]                                                               # Combine the data together
Datatable = pd.DataFrame(data,columns = ['Wreck_Name', 'Date', 'Oil_Area_m2'])                      # Package as a dataframe
Datatable                                                                                           # View data

,Wreck_Name,Date,Oil_Area_m2
0,HMS Repulse,2015-02-27,1.401008e+06


# 5 Looking through an image collection
Now we want to look at multiple images, this creates a time slider `Map.add_time_slider(imgCollection, vis_params, position='bottomright',time_interval = 3)` but jumps and slow and bit clunky.

Need to define the area we are looking at so draw a polygon around rough area of interest. Think we can come up with a better method than this. Could be a dropdown box and then zoom to the wreck etc....

In [189]:
dropdown = widgets.Dropdown(options=list(Wrecks.Wreck_ID),                                          # List of the wrecks from our data frame
                            value='HMS REPULSE',                                                    # Default value, here the repulse as one using as an example
                            description='Wreck:')                                                   # Descriptor in front of dropdown
dropdown

Dropdown(description='Wreck:', index=2, options=('SS DERBENT', 'HMS PRINCE OF WALES', 'HMS REPULSE', 'RFA WAR …

Below select the dates of the start Date and End Date of interest. Looked for example where you select the dates in one box (like when you book a hotel) but some code heavy and started taking up too much time so left it a bit clunky for now but can revist at a later date. Ideally have this/these on same row as choosing the wreck to make prettier. Again revisit at a later date.

In [221]:
StartDate = widgets.DatePicker(description='Start Date')
StartDate

DatePicker(value=None, description='Start Date')

In [222]:
EndDate = widgets.DatePicker(description='End Date')
EndDate

DatePicker(value=None, description='End Date')

In [312]:
# Loads the xls file from your personal google drive (need to do it each time to create popup correctly)
Wrecks = pd.read_excel('/content/drive/MyDrive/Wreck Database_V2.3.xls') 

MultiMap = geemap.Map()                                                                             # Base map
Lat = pd.to_numeric(Wrecks.loc[Wrecks['Wreck_ID'] == dropdown.value]['Latitude'])                   # Get the latitude of selected wreck
Lon = Wrecks.loc[Wrecks['Wreck_ID'] == dropdown.value]['Longitude']                                 # Get the longitude of selected wreck
geom = ee.Geometry.Point(Lon.iloc[0],Lat.iloc[0]);                                                  # Loction of chosen wreck
MultiMap.centerObject(geom, 9)                                                                      # Center the map on the wreck
MultiMap.add_points_from_xy(Wrecks, x="Longitude", y="Latitude")                                    # Add wreck locations
MultiMap                                                                                            # Display the map

Map(center=[3.620619000000001, 104.34518100000001], controls=(WidgetControl(options=['position', 'transparent_…

In [297]:
ImageWidget= widgets.ToggleButtons(
    options=['Reset', 'Previous Image', 'Next Image'],
    disabled=False,
    button_style='',
    tooltips=['Reset map to assess next map', 'Go back to the previous image', 'Go to the next image'],
#     icons=['check'] * 3
)




In [315]:
button = widgets.Button(description="Click Me!")
#output = widgets.Output()
ImagecontrolWidget = ipyleaflet.WidgetControl(widget=button, position='bottomright')                # Method to add to the map
MultiMap.add_control(ImagecontrolWidget)  

#display(button, output)

def on_button_clicked(b):
  with output:
    img = ee.Image('COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20150227T224655_20150227T224723_004815_005FD1_808B').select('VV')
  img_params = {'bands':'VV', 'min':-25, 'max':5}                                                     # Selects the VV band and the min and max values to display

  # The metadata doesnt have the date, have to extract from image name (called index)
  year = ee.Image(img).getString('system:index').slice(-50, 21)                                       # Select out the year
  month = ee.Image(img).getString('system:index').slice(-46, 23)                                      # Select out the month
  day = ee.Image(img).getString('system:index').slice(-44, 25)                                        # Select out the day
  date = ee.String(year.cat('-').cat(month).cat('-').cat(day))                                        # Format to date format e.g. 2023_01_01
  date = date.getInfo()                                                                               # ee.String an object on server side so need to extract it
  MultiMap.addLayer(img, img_params, date,True)  

button.on_click(on_button_clicked)

In [275]:
# Selected image from list from image collection above
img = ee.Image('COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20150227T224655_20150227T224723_004815_005FD1_808B').select('VV')
img_params = {'bands':'VV', 'min':-25, 'max':5}                                                     # Selects the VV band and the min and max values to display

# The metadata doesnt have the date, have to extract from image name (called index)
year = ee.Image(img).getString('system:index').slice(-50, 21)                                       # Select out the year
month = ee.Image(img).getString('system:index').slice(-46, 23)                                      # Select out the month
day = ee.Image(img).getString('system:index').slice(-44, 25)                                        # Select out the day
date = ee.String(year.cat('-').cat(month).cat('-').cat(day))                                        # Format to date format e.g. 2023_01_01
date = date.getInfo()                                                                               # ee.String an object on server side so need to extract it
MultiMap.addLayer(img, img_params, date,True)                                                   # Adds the image based on parameters defined


In [ ]:
Oil = (img.lt(-23.5).selfMask().rename('Oil slicks'))                                               # Selects pixel values less than -23.5
SingleImgMap.addLayer(Oil, {'palette': 'FF0000'}, 'Pixels < 23.5')                                  # Add mask layer, colour red and rename
AoI = ee.FeatureCollection(SingleImgMap.draw_features)                                              # Extracts the drawn polygon from the map
Oil_Polygons = Oil.reduceToVectors(geometry = AoI,                                                  # Polygon extracted from the map
                             scale = 10,                                                            # Resolution of data in meters
                             geometryInNativeProjection =True,                                      # Use the image projection                                 
                             maxPixels = 1e10,                                                      # Max pixels GEE will deal with                                                  
                             eightConnected =True)                                                  # Polgons will count as connect if at a diagonal
Oil_Polygons = Oil_Polygons.filter(ee.Filter.gt('count',25))                                        # Filters the polgons whic have more than 25 pixels

SingleImgMap.addLayer(ee.Image().paint(Oil_Polygons), {'palette': ['F2C80F']}, 'Oil_Polygons', True)# FE6DB6= pink

# 6 looking at satalite imagery paths

In [ ]:
OrbitsMap = geemap.Map()                                                        # Base map

geometry = ee.Geometry.Polygon(
  [[[2, 52.6],
   [2, 52.6],
   [2, 52.7],
   [2.2, 52.7],
   [2.2, 52.6]]], None, False)


# Get Sentinel-1 data for an arbitrary 12 day period
s1 = (ee.ImageCollection('COPERNICUS/S1_GRD').
  filterDate('2015-04-03', '2015-05-20').
  filterMetadata('instrumentMode', 'equals', 'IW').
  filterBounds(geometry))

# Compose an ancillary property to categorize the images in this selection

def func_xmf(f):
  return f.set('platform_relorbit',
    ee.String(f.get('platform_number')).cat('_').
    cat(ee.Number(f.get('relativeOrbitNumber_start')).format('%.0f')).
    cat('_').cat(f.get('orbitProperties_pass')))

s1 = s1.map(func_xmf)

# Check which sensor/relative orbit combinations we have
orbits = ee.Dictionary(s1.aggregate_histogram('platform_relorbit'))

keys = orbits.getInfo(); # Needed to loop

for k in keys:
  color = 'blue'
  if (k[0]=='B'):
    color = 'green'
  
  OrbitsMap.addLayer(ee.Image().paint(ee.FeatureCollection(s1).
    filterMetadata('platform_relorbit', 'equals', k), 0, 1),
    {'palette': [color]}, 'Footprint: ' + k, True)

for k in keys:
  OrbitsMap.addLayer(s1.filterMetadata('platform_relorbit', 'equals', k).first(),
    {'bands': ['angle'], 'min': 30, 'max': 45}, 'Incidence angle: ' + k, False)

OrbitsMap.addLayer(ee.Image().paint(geometry, 0, 1), {'palette': ['red']}, 'AOI', True)
OrbitsMap.centerObject(geometry, 6)
Wrecks = pd.read_excel('/content/drive/MyDrive/Wreck Database_V2.3.xls')      # Loads the xls file from your personal google drive
OrbitsMap.add_points_from_xy(Wrecks, x="Longitude", y="Latitude")
OrbitsMap

Map(center=[52.64999497288637, 2.1000000000020695], controls=(WidgetControl(options=['position', 'transparent_…

https://www.youtube.com/watch?v=UXHTiqLhsyI

remote sensing training

In [ ]:
S1